In [1]:
import xlrd
import queue
from locationiq.geocoder import LocationIQ
from math import radians, sin, cos, acos 
import json

In [2]:
class Trie:

	def __init__(self):
		self.charDic = dict()
		self.isEnd = True
		self.dest = dict()

	def insert(self,word,destination,distance):
		if(len(word)>0):
			if word[0] not in self.charDic:
				newOnj = Trie()
				newOnj.insert(word[1:],destination,distance)
				self.charDic[word[0]] = newOnj
				self.isEnd = False
			else:
				self.charDic[word[0]].insert(word[1:],destination,distance)
		else:
			self.dest[destination]=distance
			self.isEnd = True

	def getList(self,state):
		if(len(state)>0):
			return self.charDic[state[0]].getList(state[1:])
		else:
			return self.dest

In [3]:
def getGoal():
    goal = input("enter the goal State : ")
    return goal

In [4]:
def takePercept():
    initial = input("enter the start state : ")
    return initial

In [5]:
def searchFor(frontier,trie ,curState, goalState):
    path = {}
    explored = []
    path[curState] = [curState]
    while curState != goalState:
        if curState not in explored:
            for ct in trie.getList(curState).keys():
                frontier.put(ct)
                lst = list(path[curState])
                lst.append(ct)
                path[ct] = lst
            explored.append(curState)
        curState = frontier.get()
    return path[curState] 

In [6]:
def getHeauristics(source,destin):
    geocoder = LocationIQ("df1cea1ac0972a")
    src = geocoder.geocode(source)[0]
    destn = geocoder.geocode(destin)[0]
    slat = radians(float(src['lat']))
    slon = radians(float(src['lon']))
    flat = radians(float(destn['lat']))
    flon = radians(float(destn['lon']))
    return 6371.01*acos(sin(slat)*sin(flat) + cos(slat)*cos(flat)*cos(slon - flon)) 

In [7]:
def a_star_search(trie,state,goal):
	curState = state
	explored = []
	path = {}
	savedHeuristic = {}
	path[curState]=[[curState],0]
	while curState != goal:
		allNext = trie.getList(curState)
		for nextCity in allNext.keys():
			newPathLength = path[curState][1] + float(allNext[nextCity])
			if nextCity in path:
				if path[nextCity][1] > newPathLength:
					nextList = path[curState][0].copy()
					nextList.append(nextCity)
					path[nextCity].clear()
					path[nextCity].append(nextList)
					path[nextCity].append(newPathLength)
			else:
				path[nextCity]=[]
				newList = path[curState]
				pathList = newList[0].copy()
				pathList.append(nextCity)
				path[nextCity].append(pathList)
				path[nextCity].append(newPathLength)
		explored.append(curState)
		maxDist = float('inf')
		for city,dtn in path.items():
			if city not in explored:
				if city not in savedHeuristic:
					savedHeuristic[city] = getHeauristics(city,goal)
				heuristic = savedHeuristic[city]
				if dtn[1] + heuristic < maxDist:
					maxDist = dtn[1] + heuristic
					curState = city
	return path[goal][0]		
    

In [8]:
def Bi_Directional_Search(trie,state,goal):
    path1 = {}
    path2 = {}
    exploredOne=[]
    exploredTwo=[]
    queueOne = queue.Queue()
    queueTwo = queue.Queue()
    curOne = state
    curTwo = goal
    path1[curOne] = [curOne]
    path2[curTwo] = [curTwo]
    queueOne.put(curOne)
    queueTwo.put(curTwo)
    while (curOne not in exploredTwo) and (curTwo not in exploredOne):
        curOne = queueOne.get()
        curTwo = queueTwo.get()
        dictOne = trie.getList(curOne)
        dictTwo = trie.getList(curTwo)
        for key in dictOne.keys():
            if key not in exploredOne:
                queueOne.put(key)
                lst = path1[curOne].copy()  # import copy    , lst = copy.deepcopy(path1[curOne])
                lst.append(key)
                path1[key] = lst.copy()
        exploredOne.append(curOne)
        if curOne in exploredTwo:
            break
        for otherKey in dictTwo.keys():
            if otherKey not in exploredTwo:
                queueTwo.put(otherKey)
                newlst = path2[curTwo].copy()
                newlst.append(otherKey)
                path2[otherKey] = newlst.copy()
        exploredTwo.append(curTwo)
        if curTwo in exploredOne:
            break
    print("BiDIrectional : ")
    if curOne in exploredTwo:
        path2[curOne].reverse()
        path1[curOne].extend(path2[curOne][1:])
        print(path1[curOne])
    else:
        path2[curTwo].reverse()
        path1[curTwo].extend(path2[curTwo][1:])
        print(path1[curTwo])

In [9]:
class Agent:
    def __init__(self,trie):
        self.trie = trie
        self.goal = self.goalFormulation()
        self.initialState = self.percept()
        
    def goalFormulation(self):
        finalState = getGoal()
        return finalState
        
    def percept(self):
        startState = takePercept()
        return startState
        
    
    def Search(self):
        print(self.BreadthFirstSearch())
        print(self.AstarSearch())
        self.BiDirectionalSearch()
        print(self.DepthFirstSearch())
        
    def BreadthFirstSearch(self):
        print("BFS : ")
        return searchFor(queue.Queue(),self.trie,self.initialState,self.goal)
        
    def AstarSearch(self):
        print("Astar : ")
        return a_star_search(self.trie,self.initialState,self.goal)
    
    def DepthFirstSearch(self):
        print("DFS : ")
        return searchFor(queue.LifoQueue(),self.trie,self.initialState,self.goal)
    
    def BiDirectionalSearch(self):
        return Bi_Directional_Search(self.trie,self.initialState,self.goal)

In [10]:
trie = Trie()
loc="Indian_capitals.xlsx"
wb=xlrd.open_workbook(loc)
sheet=wb.sheet_by_index(0)
for i in range(0,sheet.nrows):
    trie.insert(sheet.cell_value(i,0),sheet.cell_value(i,1),sheet.cell_value(i,2))
    trie.insert(sheet.cell_value(i,1),sheet.cell_value(i,0),sheet.cell_value(i,2))    

In [ ]:
agent = Agent(trie)
agent.Search()